In [1]:
import numpy as np
import pandas as pd
import string
import re

In [2]:
with open('documents.txt',mode='r',encoding='utf8',errors='ignore') as f:
    text = f.read()

In [3]:
data = list(map(lambda x:x.split('\t'),text.split('\n')))

In [4]:
df = pd.DataFrame(data[1:],columns=data[0])

In [5]:
df.head(10)

,id,text,tag
0,000.jpeg,,O
1,000.jpeg,.,O
2,000.jpeg,040-4852,B-PHONE
3,000.jpeg,"""8881,""",I-PHONE
4,000.jpeg,90309,B-PHONE
5,000.jpeg,52549,I-PHONE
6,000.jpeg,Fi,O
7,000.jpeg,/laurelsoverseaseducation,O
8,000.jpeg,@:,O
9,000.jpeg,LAURELS,B-ORG


In [6]:
print(text)

id	text	tag
000.jpeg	 	O
000.jpeg	.	O
000.jpeg	040-4852	B-PHONE
000.jpeg	"8881,"	I-PHONE
000.jpeg	90309	B-PHONE
000.jpeg	52549	I-PHONE
000.jpeg	Fi	O
000.jpeg	/laurelsoverseaseducation	O
000.jpeg	@:	O
000.jpeg	LAURELS	B-ORG
000.jpeg	OVERSEAS	I-ORG
000.jpeg	EDUCATIONAL	I-ORG
000.jpeg	CONSULTANCY	I-ORG
000.jpeg	PVT.	I-ORG
000.jpeg	LTD.	I-ORG
000.jpeg	Sea	O
000.jpeg	|	O
000.jpeg	U.K	O
000.jpeg	AUSTRALIA	O
000.jpeg	CANADA	O
000.jpeg	IRELAND	O
000.jpeg	 	O
000.jpeg	 	O
000.jpeg	 	O
000.jpeg	 	O
000.jpeg	 	O
000.jpeg	 	O
000.jpeg	www.laurelseducation.com	B-WEB
000.jpeg	)%info@laurelseducation.com	B-EMAIL
000.jpeg	 	O
001.jpeg	john	B-NAME
001.jpeg	smith	I-NAME
001.jpeg	marketing	B-DES
001.jpeg	manager	I-DES
001.jpeg	web:	O
001.jpeg	www.psdgraphics.com	B-WEB
001.jpeg	phone:	O
001.jpeg	123-456-7890	B-PHONE
001.jpeg	mail:	O
001.jpeg	email@psdgraphics.com	B-EMAIL
002.jpeg	    	O
002.jpeg	   	O
002.jpeg	Sau	O
002.jpeg	0	O
002.jpeg	98489	B-PHONE
002.jpeg	24441	I-PHONE
002.jpeg	dy	O
002.jpeg	"08672,"

In [7]:
whitespace = string.whitespace
punctuation = "!#$%&\'()*+:;<=>?[\\]^`{|}~.,"
tableWhitespace = str.maketrans('','',whitespace)
tablePunctuation = str.maketrans('','',punctuation)
def cleanText(txt):
    text = str(txt)
    text = text.lower()
    removewhitespace = text.translate(tableWhitespace)
    removepunctuation = removewhitespace.translate(tablePunctuation)
    
    return str(removepunctuation)

In [8]:
df['text'] = df['text'].apply(cleanText)

In [9]:
dataClean = df.query("text != '' ")
dataClean.dropna(inplace=True)

C:\Users\Bala\AppData\Local\Temp\ipykernel_18136\3379659824.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataClean.dropna(inplace=True)


In [13]:
dataClean.head(10)

,id,text,tag
2,000.jpeg,040-4852,B-PHONE
3,000.jpeg,"""8881""",I-PHONE
4,000.jpeg,90309,B-PHONE
5,000.jpeg,52549,I-PHONE
6,000.jpeg,fi,O
7,000.jpeg,/laurelsoverseaseducation,O
8,000.jpeg,@,O
9,000.jpeg,laurels,B-ORG
10,000.jpeg,overseas,I-ORG
11,000.jpeg,educational,I-ORG


In [11]:
print(dataClean)

             id      text      tag
2      000.jpeg  040-4852  B-PHONE
3      000.jpeg    "8881"  I-PHONE
4      000.jpeg     90309  B-PHONE
5      000.jpeg     52549  I-PHONE
6      000.jpeg        fi        O
...         ...       ...      ...
10137  280.jpeg    patent        O
10138  280.jpeg   5722439        O
10139  280.jpeg     other        O
10140  280.jpeg   patents        O
10141  280.jpeg   pending        O

[8148 rows x 3 columns]


In [14]:
group = dataClean.groupby(by='id')

In [15]:
documents = group.groups.keys()

In [16]:
allDocumentsData = []
for document in documents:
    documentData = []
    grouparray = group.get_group(document)[['text','tag']].values
    content = ''
    annotations = {'entities':[]}
    start = 0
    end = 0
    for text, label in grouparray:
        text = str(text)
        stringLength = len(text) + 1

        start = end
        end = start + stringLength

        if label != 'O':
            annot = (start,end-1,label)
            annotations['entities'].append(annot)

        content = content + text + ' '
        
        
    documentData = (content,annotations)
    allDocumentsData.append(documentData)

In [23]:
content

'kenneth y "fishman" dds steven e "roth" dmd timothy "chase" dmd 220 east 63rd street new "york" ny 10065 s milesn y 212-752-6537 cosmetic implant dentistry wwwsmilesnycom """creating" new yorks most beautiful "smiles""" flosstard 12 varps patent 5722439 other patents pending '

In [24]:
annotations

{'entities': [(0, 7, 'B-NAME'),
  (8, 9, 'I-NAME'),
  (116, 128, 'B-PHONE'),
  (129, 137, 'B-ORG'),
  (138, 145, 'I-ORG'),
  (156, 170, 'B-EMAIL')]}

In [17]:
allDocumentsData

[('040-4852 "8881" 90309 52549 fi /laurelsoverseaseducation @ laurels overseas educational consultancy pvt ltd sea uk australia canada ireland wwwlaurelseducationcom info@laurelseducationcom ',
  {'entities': [(0, 8, 'B-PHONE'),
    (9, 15, 'I-PHONE'),
    (16, 21, 'B-PHONE'),
    (22, 27, 'I-PHONE'),
    (59, 66, 'B-ORG'),
    (67, 75, 'I-ORG'),
    (76, 87, 'I-ORG'),
    (88, 99, 'I-ORG'),
    (100, 103, 'I-ORG'),
    (104, 107, 'I-ORG'),
    (140, 162, 'B-WEB'),
    (163, 187, 'B-EMAIL')]}),
 ('john smith marketing manager web wwwpsdgraphicscom phone 123-456-7890 mail email@psdgraphicscom ',
  {'entities': [(0, 4, 'B-NAME'),
    (5, 10, 'I-NAME'),
    (11, 20, 'B-DES'),
    (21, 28, 'I-DES'),
    (33, 50, 'B-WEB'),
    (57, 69, 'B-PHONE'),
    (75, 95, 'B-EMAIL')]}),
 ('sau 0 98489 24441 dy "08672" 224441 /enkateswapa wie ',
  {'entities': [(6, 11, 'B-PHONE'), (12, 17, 'I-PHONE'), (36, 48, 'B-ORG')]}),
 ('prasad @ "99631735359490400000" i flex design album design visiting cards dtp 

In [21]:
content.find('australia')

-1

In [25]:
import random

In [26]:
random.shuffle(allDocumentsData)

In [27]:
len(allDocumentsData)

261

In [19]:
TrainData = allDocumentsData[:230]
TestData = allDocumentsData[230:]

In [20]:
import pickle

In [21]:
pickle.dump(TrainData,open('./data/TrainData.pickle',mode='wb'))
pickle.dump(TestData,open('./data/TestData.pickle',mode='wb'))